# NLP features (sentiment, length, etc.)

In [14]:
#Import the saved cleaned data 
import pandas as pd

df = pd.read_csv("../data/processed/cleaned.csv")


In [15]:
# Only needed if src isn't being found (depends on your VSCode setup)
import sys
sys.path.append("..")  # Go one directory up so Python sees "src" folder

# Imports
from src.features.basic_features import (
    add_char_count,
    add_has_tsla,
    add_has_emoji,
    add_exclam_count,
    add_is_reply
)


In [16]:
#apply the imported features to the data frame

features_df = df.copy()
features_df = add_char_count(features_df)
features_df = add_has_tsla(features_df)
features_df = add_has_emoji(features_df)
features_df = add_exclam_count(features_df)
features_df = add_is_reply(features_df)


/home/james/finance_job/NLP_market_reaction/src/features/basic_features.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["has_tsla"] = df["text"].str.contains(pattern, case=False, regex=True).astype(int)


In [18]:
features_df[["text", "char_count", "has_tsla", "has_emoji", "exclam_count", "is_reply"]].head()

,text,char_count,has_tsla,has_emoji,exclam_count,is_reply
0,@BillyM2k I find the gold toe sock – inevitabl...,136,0,0,0,1
1,"Sock Con, the conference for socks",34,0,0,0,0
2,Always something new for the magazine cover an...,89,0,0,0,0
3,@ExplainThisBob This guy gets it,32,0,0,0,1
4,Sock tech is so advanced that you can get pret...,87,0,0,1,0


In [19]:
features_df.to_csv("../data/processed/cleaned_with_features.csv", index=False)

In [28]:
import nltk
nltk.download("vader_lexicon")
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/james/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [31]:
def add_vader_sentiment(df):
    sia = SentimentIntensityAnalyzer()
    df["vader_sentiment"] = df["text"].apply(lambda x: sia.polarity_scores(x)["compound"])
    return df


In [33]:
vader_features_df = add_vader_sentiment(features_df)
vader_features_df.to_csv("../data/processed/vader_features.csv", index=False)

In [34]:
#Applying FinBERT

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

# Load FinBERT model
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create pipeline
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Apply to a single tweet
result = finbert("Tesla is going to the moon 🚀")
print(result)


/home/james/finance_job/NLP_market_reaction/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'